In [1]:
import os
import pandas as pd
import glob
import menpo.io as mio
import numpy as np
#np.random.seed(1337)  # for reproducibility
from keras.utils import np_utils
from keras import backend as K

Using Theano backend.


In [2]:
# returns (X_train, y_train), (X_test, y_test)
def randomSplit(X, y, propTest):
    assert X.shape[0] == y.shape[0]
    p = np.random.permutation(len(y))
    sX, sy = X[p], y[p]
    cutoff = round(propTest * len(y))
    return (sX[cutoff:], sy[cutoff:]), (sX[0:cutoff], sy[0:cutoff])

def randomOrder(X, y):
    p = np.random.permutation(len(y))
    return X[p], y[p]

In [3]:
data_file = 'gold.tsv'
image_folder = './cropped/' # must end with '/'
variable = 'ageC'# relevant column of data_file
height = 128
width = 128
channels = 3

In [6]:
annos = pd.DataFrame.from_csv(data_file, header=0, index_col=None, sep="\t")

ids = list(set(annos['subject_id'].get_values()))
ids = np.random.permutation(ids)
boundary = round(0.9 * len(ids))
train_ids = set(ids[0:boundary])
test_ids = set(ids[boundary:])

train_images = []
train_labels = []
test_images = []
test_labels = []
for fp in glob.glob(image_folder + '*.jpg'):
    fn = os.path.basename(fp)
    image = mio.import_image(fp)
    assert(image.shape == (height, width))
    assert(image.n_channels == channels)
    if bool(np.random.randint(0,1)):
        image = image.mirror()
    if annos.loc[annos['file_name'] == fn]['subject_id'].sum() in train_ids:
        train_images.append(image)
        train_labels.append(annos.loc[annos['file_name'] == fn][variable].sum())
    else:
        test_images.append(image)
        test_labels.append(annos.loc[annos['file_name'] == fn][variable].sum())


In [12]:
nb_classes = len(set(train_labels + test_labels))

# input image dimensions
img_rows, img_cols, img_channels = height, width, channels

X_train = np.stack([img.pixels for img in train_images])
X_test = np.stack([img.pixels for img in test_images])
if K.image_dim_ordering() == 'tf':
    X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, channels)
    X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, channels)
    input_shape = input_shape = (img_rows, img_cols, channels)
else:
    input_shape = (channels, img_rows, img_cols)
    
y_train = np.array(train_labels)
y_test = np.array(test_labels)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

# convert class vectors to n-ary class matrices
y_train = np_utils.to_categorical(y_train, nb_classes)
y_test = np_utils.to_categorical(y_test, nb_classes)

print('y_train shape:', y_train.shape)

X_train, y_train = randomOrder(X_train, y_train)
X_test, y_test = randomOrder(X_test, y_test)

X_train shape: (6692, 128, 128, 3)
6692 train samples
718 test samples
y_train shape: (6692, 5)


In [13]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D

batch_size = 50
nb_epoch = 2

# number of convolutional filters to use
nb_filters1 = 48
# convolution kernel size
kernel1 = (7, 7)
# size of pooling area for max pooling
pool1 = (3, 3)

# number of convolutional filters to use
nb_filters2 = 128
# convolution kernel size
kernel2 = (5, 5)
# size of pooling area for max pooling
pool2 = (3, 3)

# number of convolutional filters to use
nb_filters3 = 192
# convolution kernel size
kernel3 = (3, 3)
# size of pooling area for max pooling
pool3 = (3, 3)

dense1 = 256
dense2 = 256

model = Sequential()

model.add(Convolution2D(nb_filters1, kernel1[0], kernel1[1],
                        border_mode='valid',
                        input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=pool1))
model.add(Dropout(0.2))
model.add(Convolution2D(nb_filters2, kernel2[0], kernel2[1]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=pool2))
model.add(Dropout(0.2))
model.add(Convolution2D(nb_filters3, kernel3[0], kernel3[1]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=pool3))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(dense1))
model.add(Activation('relu'))
model.add(Dropout(0.4))
model.add(Dense(dense2))
model.add(Activation('relu'))
model.add(Dropout(0.4))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])

model.fit(X_train, y_train, batch_size=batch_size, nb_epoch=nb_epoch,
          verbose=1, validation_data=(X_test, y_test))
score = model.evaluate(X_test, y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

Train on 6692 samples, validate on 718 samples
Epoch 1/2
6692/6692 [==============================] - 793s - loss: 1.4585 - acc: 0.3646 - val_loss: 1.3993 - val_acc: 0.4903
Epoch 2/2
6692/6692 [==============================] - 776s - loss: 1.4328 - acc: 0.3772 - val_loss: 1.3729 - val_acc: 0.4903
Test score: 1.37290061815
Test accuracy: 0.490250696213


In [14]:
model.save("age_classifier.h5")